# Library Managemnet System

This project implements a Library Management System using Object-Oriented Programming (OOP) concepts in Python. It consists of three main classes: Book, Member, and Library. The Book class manages details such as title, author, ISBN, and availability status (available or borrowed). Meanwhile, the Member class handles member information including name, unique ID, and their borrowed book list. These classes are designed to efficiently handle interactions within the library environment. To ensure data persistence, I've Implemented JSON serialization. This technique converts Python objects (books and members) into JSON format for seamless storage and retrieval from files.

In [1]:
class Book:
    def __init__(self, title, author, isbn, status="available"):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.status = status

    def display_info(self):
        print(f"Title: {self.title}, Author: {self.author}, ISBN: {self.isbn}, Status: {self.status}")

    def mark_as_borrowed(self):
        self.status = "borrowed"

    def mark_as_returned(self):
        self.status = "available"


In [2]:
class Member:
    def __init__(self, name, member_id, borrowed_books=None):
        self.name = name
        self.member_id = member_id
        self.borrowed_books = borrowed_books if borrowed_books else []

    def borrow_book(self, book):
        if book.status == "available":
            book.mark_as_borrowed()
            self.borrowed_books.append(book)
            print(f"{self.name} has borrowed '{book.title}'.")
        else:
            print(f"'{book.title}' is currently not available.")

    def return_book(self, book):
        if book in self.borrowed_books:
            book.mark_as_returned()
            self.borrowed_books.remove(book)
            print(f"{self.name} has returned '{book.title}'.")
        else:
            print(f"{self.name} did not borrow '{book.title}'.")

    def display_info(self):
        print(f"Member ID: {self.member_id}, Name: {self.name}")
        if self.borrowed_books:
            print("Borrowed books:")
            for book in self.borrowed_books:
                print(f" - {book.title}")
        else:
            print("No borrowed books.")

    def to_dict(self):
        return {
            "name": self.name,
            "member_id": self.member_id,
            "borrowed_books": [book.__dict__ for book in self.borrowed_books]
        }

    @classmethod
    def from_dict(cls, data):
        borrowed_books = [Book(**book_data) for book_data in data["borrowed_books"]]
        return cls(data["name"], data["member_id"], borrowed_books)


In [3]:
import json

class Library:
    def __init__(self, books_file="books.json", members_file="members.json"):
        self.books_file = books_file
        self.members_file = members_file
        self.books = self.load_books()
        self.members = self.load_members()
        if not self.books:
            self.add_initial_books()

    def load_books(self):
        try:
            with open(self.books_file, "r") as file:
                books_data = json.load(file)
                return [Book(**data) for data in books_data]
        except (FileNotFoundError, json.JSONDecodeError):
            return []

    def save_books(self):
        with open(self.books_file, "w") as file:
            books_data = [book.__dict__ for book in self.books]
            json.dump(books_data, file, indent=4)

    def load_members(self):
        try:
            with open(self.members_file, "r") as file:
                members_data = json.load(file)
                return [Member.from_dict(data) for data in members_data]
        except (FileNotFoundError, json.JSONDecodeError):
            return []

    def save_members(self):
        with open(self.members_file, "w") as file:
            members_data = [member.to_dict() for member in self.members]
            json.dump(members_data, file, indent=4)

    def add_book(self, book):
        self.books.append(book)
        self.save_books()
        print(f"Book '{book.title}' added to the library.")

    def register_member(self, member):
        self.members.append(member)
        self.save_members()
        print(f"Member '{member.name}' registered with ID {member.member_id}.")

    def issue_book(self, book_title, member_id):
        member = next((m for m in self.members if m.member_id == member_id), None)
        book = next((b for b in self.books if b.title == book_title), None)

        if member and book:
            member.borrow_book(book)
            self.save_books()
            self.save_members()
        else:
            print("Member or Book not found.")

    def return_book(self, book_title, member_id):
        member = next((m for m in self.members if m.member_id == member_id), None)
        book = next((b for b in self.books if b.title == book_title), None)

        if member and book:
            member.return_book(book)
            self.save_books()
            self.save_members()
        else:
            print("Member or Book not found.")

    def display_all_books(self):
        for book in self.books:
            book.display_info()

    def display_all_members(self):
        for member in self.members:
            member.display_info()

    def add_initial_books(self):
        initial_books = [
            Book("Umrao Jaan Ada", "Mirza Hadi Ruswa", "9789693514419"),
            Book("Aangan", "Khadija Mastoor", "9789693514426"),
            Book("Aag Ka Darya", "Qurratulain Hyder", "9789693514433"),
            Book("Ghubar-e-Khatir", "Abul Kalam Azad", "9789693514440"),
            Book("Zavia", "Ashfaq Ahmed", "9789693514457"),
            Book("Toba Tek Singh", "Saadat Hasan Manto", "9789693514464"),
            Book("Khuda Ki Basti", "Shaukat Siddiqui", "9789693514471"),
            Book("Raja Gidh", "Bano Qudsia", "9789693514488"),
            Book("Ali Pur Ka Aili", "Mumtaz Mufti", "9789693514495"),
            Book("Shahab Nama", "Qudrat Ullah Shahab", "9789693514501")
        ]

        self.books.extend(initial_books)
        self.save_books()
        print("Initial Urdu literature books have been added to the library.")


In [4]:
def main_menu():
    library = Library()

    while True:
        print("\nLibrary Management System")
        print("1. Display all books")
        print("2. Display all members")
        print("3. Issue a book")
        print("4. Return a book")
        print("5. Add a book")
        print("6. Register a member")
        print("7. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            print("\nAll books in the library:")
            library.display_all_books()
        elif choice == "2":
            print("\nAll members in the library:")
            library.display_all_members()
        elif choice == "3":
            book_title = input("Enter the title of the book to issue: ")
            member_id = int(input("Enter the member ID: "))
            library.issue_book(book_title, member_id)
            member = next((m for m in library.members if m.member_id == member_id), None)
            if member:
                print("\nMember info after borrowing a book:")
                member.display_info()
        elif choice == "4":
            book_title = input("Enter the title of the book to return: ")
            member_id = int(input("Enter the member ID: "))
            library.return_book(book_title, member_id)
            member = next((m for m in library.members if m.member_id == member_id), None)
            if member:
                print("\nMember info after returning a book:")
                member.display_info()
        elif choice == "5":
            title = input("Enter the title of the book: ")
            author = input("Enter the author of the book: ")
            isbn = input("Enter the ISBN of the book: ")
            book = Book(title, author, isbn)
            library.add_book(book)
        elif choice == "6":
            name = input("Enter the name of the member: ")
            member_id = int(input("Enter the member ID: "))
            member = Member(name, member_id)
            library.register_member(member)
        elif choice == "7":
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main_menu()



Library Management System
1. Display all books
2. Display all members
3. Issue a book
4. Return a book
5. Add a book
6. Register a member
7. Exit


Enter your choice:  1



All books in the library:
Title: Umrao Jaan Ada, Author: Mirza Hadi Ruswa, ISBN: 9789693514419, Status: available
Title: Aangan, Author: Khadija Mastoor, ISBN: 9789693514426, Status: borrowed
Title: Aag Ka Darya, Author: Qurratulain Hyder, ISBN: 9789693514433, Status: available
Title: Ghubar-e-Khatir, Author: Abul Kalam Azad, ISBN: 9789693514440, Status: available
Title: Zavia, Author: Ashfaq Ahmed, ISBN: 9789693514457, Status: available
Title: Toba Tek Singh, Author: Saadat Hasan Manto, ISBN: 9789693514464, Status: available
Title: Khuda Ki Basti, Author: Shaukat Siddiqui, ISBN: 9789693514471, Status: available
Title: Raja Gidh, Author: Bano Qudsia, ISBN: 9789693514488, Status: available
Title: Ali Pur Ka Aili, Author: Mumtaz Mufti, ISBN: 9789693514495, Status: available
Title: Shahab Nama, Author: Qudrat Ullah Shahab, ISBN: 9789693514501, Status: available
Title: Janat ky patay, Author: Nemrah Ahmed, ISBN: 844746446764, Status: available

Library Management System
1. Display all boo

Enter your choice:  2



All members in the library:

Library Management System
1. Display all books
2. Display all members
3. Issue a book
4. Return a book
5. Add a book
6. Register a member
7. Exit


...

Enter your choice:  3
Enter the title of the book to issue:  janat ky patay
Enter the member ID:  4


Member or Book not found.

Member info after borrowing a book:
Member ID: 4, Name: Ahmad
No borrowed books.

Library Management System
1. Display all books
2. Display all members
3. Issue a book
4. Return a book
5. Add a book
6. Register a member
7. Exit


Enter your choice:  2



All members in the library:
Member ID: 1, Name: Emama
No borrowed books.
Member ID: 2, Name: Hamna
Borrowed books:
 - Zavia
Member ID: 3, Name: iman
No borrowed books.
Member ID: 4, Name: Ahmad
No borrowed books.

Library Management System
1. Display all books
2. Display all members
3. Issue a book
4. Return a book
5. Add a book
6. Register a member
7. Exit


Enter your choice:  7
